In [1]:
import pdfplumber
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS


from dotenv import load_dotenv
import os

# Load the .env file located in the project directory
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [4]:

# Change this to the local path where your PDF file is located.
local_pdf_path = "PDF/完整給付規定1120424.pdf"

with pdfplumber.open(local_pdf_path) as pdf:
    raw_text = ''
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        if text:
            raw_text += text

# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 
text_splitter = RecursiveCharacterTextSplitter(
    separators=["。", "！", "？", "\n", " "],
    chunk_size=800,
    chunk_overlap=50,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)


In [8]:
import pinecone
embeddings = OpenAIEmbeddings()

# initialize pinecone
PINECONE_API_ENV = 'asia-southeast1-gcp-free'
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "nhi" # put in the name of your pinecone index here

docsearch = Pinecone.from_texts(texts, embeddings, index_name=index_name)


In [2]:
import pinecone
embeddings = OpenAIEmbeddings()

# initialize pinecone
PINECONE_API_ENV = 'asia-southeast1-gcp-free'
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "nhi" # put in the name of your pinecone index here

docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

/Users/kuoyihsin/opt/anaconda3/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [48]:
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

def answer_question(docs, user_question, useAzure = False):
    # add Azure gpt-4 api support in the future
    if useAzure:
        load_dotenv()
        os.environ["OPENAI_API_TYPE"] = "azure"
        os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
        os.environ["OPENAI_API_BASE"] = "https://user1-create-gpt.openai.azure.com/" 
        os.environ["OPENAI_API_KEY"] = os.getenv("AZURE_API_KEY")
        chat = AzureChatOpenAI(deployment_name="gpt-4", openai_api_version="2023-03-15-preview")
        # chat = AzureChatOpenAI(deployment_name="gpt-35-turbo", openai_api_version="2023-03-15-preview")
    else:
        chat = ChatOpenAI(temperature=0.1)

    messages = [SystemMessage(
        content="You are a helpful bot. If needed, you can use the following document to help you answer the question.")]

    # Add the relevant documents to the chat context
    for i, doc in enumerate(docs):
        messages.append(SystemMessage(content=f"Document {i+1}: {doc}"))

    messages.append(HumanMessage(content=user_question))

    response = chat(messages).content
    messages.append(AIMessage(content=response))

    return response

In [24]:
answer_docs

[Document(page_content='2.急性 Q 波心肌梗塞於症狀發生十二小時內，於執行 PTCA 時得併用\neptifibatide。\n2.1.1.8.Clopidogrel-acetylsalicylic acid（如 CoPlavix）：(100/9/1、\n101/12/1)\n1.置放金屬支架3個月內得使用；置放塗藥支架6個月內得使用。需於病歷註明支\n架置放手術之日期。(100/9/1、101/12/1)\n2.置放金屬支架後發生支架內再狹窄而施行藥物釋放型冠狀動脈氣球導管者，1\n個月內得使用；置放塗藥支架後發生支架內再狹窄而施行藥物釋放型冠狀動脈\n氣球導管者，使用期間可再延長1個月。(101/12/1)\n3.用於已發作之急性冠心症(不穩定性心絞痛和心肌梗塞)而住院的病人時，最長\n9個月。需於病歷註明住院時間。（100/9/1）\n2.1.1.9.Ticagrelor（如 Brilinta）：(102/7/1)\n限用於已發作之急性冠心症(包括不穩定型心絞痛、非 ST 段上升型心肌梗塞或\nST 段上升型心肌梗塞)而住院的病人時，得與 acetylsalicylic acid (如\nAspirin) 合併治療，最長9個月。需於病歷註明住院時間。\n2.1.1.10. Prasugrel（如 Efient）：(107/11/1)\n限用於需要冠狀動脈介入性治療(PCI)的已發作之急性冠心症(包括不穩定型心\n絞痛、非 ST 段上升型心肌梗塞或 ST 段上升型心肌梗塞)而住院的病人時，得\n與 acetylsalicylic acid (如 Aspirin) 合併治療，最長12個月。需於病歷註\n明住院時間', metadata={}),
 Document(page_content='(2)第1日至21日，每日早晚各一次，每次限用一粒。\n(3)第22日起，每日一次，每次限用一粒，每6個月評估一次。\n4.Rivaroxaban 2.5mg 與 aspirin 併用，可用於發生缺血事件高危險族群之冠狀\n動脈(CAD)或症狀性周邊動脈疾病(PAD)病人，以預防動脈粥狀硬化血栓形成事\n件(Atherothrombotic events)：(110/7/1)\n(1)高風險的冠狀動脈硬化須符合：2年內發生2次(含)以上心血管事件(如

In [29]:
query = "clopidogrel"
answer_docs = docsearch.similarity_search(query, include_metadata=True)
question = "clopidogrel+aspirin的健保給付規定"
response = answer_question(answer_docs[:2], question)
print(response)

根據文件1，clopidogrel+aspirin的健保給付規定如下：

1. 置放金屬支架3個月內得使用；置放塗藥支架6個月內得使用。需於病歷註明支架置放手術之日期。
2. 置放金屬支架後發生支架內再狹窄而施行藥物釋放型冠狀動脈氣球導管者，1個月內得使用；置放塗藥支架後發生支架內再狹窄而施行藥物釋放型冠狀動脈氣球導管者，使用期間可再延長1個月。
3. 用於已發作之急性冠心症(不穩定性心絞痛和心肌梗塞)而住院的病人時，最長9個月。需於病歷註明住院時間。

需要注意的是，以上規定僅供參考，實際使用時應遵循醫師的處方和指示。


In [5]:
query = '''clopidogrel'''
answer_docs = docsearch.similarity_search(query, include_metadata=True)
question = '''請根據clopidogrel的健保給付規定，檢視是否不符合給付規定，
描述如下：今日是112/05/12，病人於111/05/12放置的金屬支架'''
response = answer_question(answer_docs[:2], question)
print(response)

根據文件1，放置金屬支架後發生支架內再狹窄而施行藥物釋放型冠狀動脈氣球導管者，1個月內得使用；置放塗藥支架後發生支架內再狹窄而施行藥物釋放型冠狀動脈氣球導管者，使用期間可再延長1個月。因此，如果病人於111/05/12放置金屬支架後發生支架內再狹窄而施行藥物釋放型冠狀動脈氣球導管，則在112/06/12之前使用clopidogrel是符合健保給付規定的。但如果病人的情況不符合上述規定，則使用clopidogrel可能不符合健保給付規定。


In [28]:
query = '''clopidogrel'''
answer_docs = docsearch.similarity_search(query, include_metadata=True)
question = '''請根據clopidogrel併用aspirin的健保給付規定來回答以下問題
病人於2022/05/12放置的金屬支架，
於2022/10/22發生支架內再狹窄而施行藥物釋放型冠狀動脈氣球導管者，
請問他總共在哪些時候可以併用clopidogrel+aspirin
'''
response = answer_question(answer_docs[:2], question)
print(response)

根據健保給付規定，放置金屬支架後發生支架內再狹窄而施行藥物釋放型冠狀動脈氣球導管者，使用clopidogrel-acetylsalicylic acid的期間可以再延長1個月。因此，病人可以在2022/11/22前繼續使用clopidogrel-acetylsalicylic acid。


In [49]:
question = '''請根據clopidogrel+aspirin的健保給付規定來回答以下問題
病人於2022/05/12放置的金屬支架，
且又於2022/10/22發生支架內再狹窄而施行冠狀動脈氣球導管，
請列出他能使用clopidogrel+aspirin的所有時間段
'''
response = answer_question(answer_docs[:2], question, useAzure =True)
print(response)

根據提供的文件，Clopidogrel-acetylsalicylic acid（如 CoPlavix）的健保給付規定如下：

1. 置放金屬支架後的3個月內可使用。病人於2022/05/12放置金屬支架，因此可使用Clopidogrel+Aspirin的時間段為2022/05/12至2022/08/11。

2. 置放金屬支架後發生支架內再狹窄而施行藥物釋放型冠狀動脈氣球導管者，1個月內可使用。病人於2022/10/22發生支架內再狹窄並施行冠狀動脈氣球導管，因此可使用Clopidogrel+Aspirin的時間段為2022/10/22至2022/11/21。

綜合以上，病人能使用Clopidogrel+Aspirin的所有時間段為：

1. 2022/05/12至2022/08/11
2. 2022/10/22至2022/11/21
